In [61]:
import os
from azure.core.credentials import AzureKeyCredential
from langchain.schema import HumanMessage
from langchain.chat_models import AzureChatOpenAI
os.environ["AZURE_OPENAI_API_KEY"]  = 'b82effcf491e45a088b1cd578713311c'
os.environ["OPENAI_API_VERSION"]    = '2023-05-15'
os.environ["AZURE_OPENAI_ENDPOINT"] = 'https://usesharedaopenai001.openai.azure.com/'
os.environ["OPENAI_API_TYPE"]       = 'azure'
# Variables-------------------------------------------------------
index                               = "azure-cognitive-search-vector-demo"
azure_search_endpoint               = 'https://genai0.search.windows.net'
MODEL                               = "gtp35turbo-latest"
key                                 = 'lvhCA67EeE3JRyxyem5L0wGJSfOxscm2jft887ECdJAzSeDzoCNZ'
model                               = "text-embedding-ada-002"
credential                          = AzureKeyCredential(key)
COMPLETION_TOKENS                   = 1000
top_search_vector_k                 = 5

In [62]:
llm = AzureChatOpenAI(model_name=MODEL, temperature=0)

In [63]:
from langchain import PromptTemplate

In [66]:
# create the prompt, here we use multiple inputs
prompt = PromptTemplate(
    template=""""/
    You are a helpful Name Entity Recognition System. From the following text:    
    
    <<{text}>>
    
    Extract this target word class: '{word_class}'. Return the extracted element in a list. 
    For word class related to dates or years always return it in the format 'YYYY-MM-DD'.
    For intervals with a start-date and end-date return both dates in a list. 
    For word class 'company_name' return it in capital letters.
    """,
    input_variables=["word_class", "text"]
)

#query = "What is the revenue of Pfizer from july 2020 to october 2021"
query = "In agreement with the information outlined in the income statement, what is the FY2015 - FY2017 3 year average net profit margin (as a %) for Best Buy? Answer in units of percents and round to one decimal place."
word_class = "dates"
#word_class = "company_name"

# format the prompt to add variable values
prompt_formatted_str: str = prompt.format(
    word_class = word_class,
    text = query,
    )

In [67]:
# make a prediction
prediction = llm.predict(prompt_formatted_str)

# print the prediction
print(prediction)

['2015-01-01', '2017-12-31']
